In [1]:
'''import cv2
import os
from ultralytics import YOLO

def load_model(model_path):
    """Load the trained YOLO model"""
    return YOLO(model_path)

def detect_objects(model, image_path):
    """Detect objects in the image using the YOLO model"""
    image = cv2.imread(image_path)
    results = model(image)
    return results, image  # Return both detection results and imag

def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

def draw_detections(image, propeller_boxes, floor_box):
    """Draw bounding boxes around detected propellers and floor"""
    for (x_min, y_min, x_max, y_max) in propeller_boxes:
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green for propellers
        cv2.putText(image, "Propeller", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    if floor_box:
        x_min, y_min, x_max, y_max = floor_box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Red for floor
        cv2.putText(image, "Floor", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    return image

# Example usage
model_floor_path = "best (5).pt"  # Model detecting floor, ceiling, wall
model_propeller_path = "best (9).pt"  # Model detecting propellers
image_path = "Propellers test/IMG_3103.JPEG"  # Input image
output_folder = "Detected_Images"  # Folder to save images
os.makedirs(output_folder, exist_ok=True)  # Create folder if not exists

# Load YOLO models
floor_model = load_model(model_floor_path)
propeller_model = load_model(model_propeller_path)

# Detect floor using model best(5).pt
floor_results, image = detect_objects(floor_model, image_path)
floor_y_top, floor_box = get_floor_coordinates(floor_results)

# Detect propellers using model best(9).pt
propeller_results, _ = detect_objects(propeller_model, image_path)
propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)

if not propeller_boxes or floor_y_top is None:
    print(f"No propellers or floor detected in {image_path}")
else:
    print(f"\nImage: {image_path}")

    # Draw detected objects on image
    output_image = draw_detections(image, propeller_boxes, floor_box)

    # Display the output image
    cv2.imshow("Detected Objects", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, output_image)
    print(f"Output saved at: {output_path}")'''

'import cv2\nimport os\nfrom ultralytics import YOLO\n\ndef load_model(model_path):\n    """Load the trained YOLO model"""\n    return YOLO(model_path)\n\ndef detect_objects(model, image_path):\n    """Detect objects in the image using the YOLO model"""\n    image = cv2.imread(image_path)\n    results = model(image)\n    return results, image  # Return both detection results and imag\n\ndef get_floor_coordinates(results):\n    """Extract the top Y-coordinate and bounding box of the floor"""\n    floor_y_top = None\n    floor_box = None\n    \n    for result in results:\n        for box in result.boxes:\n            class_id = int(box.cls.cpu().numpy().item())  # Get class ID\n            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box\n            \n            if class_id == 1:  # Assuming class 1 is "floor"\n                floor_y_top = y_min  # Top Y-coordinate of floor\n                floor_box = (x_min, y_min, x_max, y_max)\n    \n    return floor_y_top, floor

In [2]:
import cv2
import torch
from ultralytics import YOLO

def detect_coke_bottle(image_path, model_path, real_height_cm=24):
    """
    Detects a  Coca-Cola bottle and calculates its real-world height.
    :param image_path: Path to the image.
    :param model_path: Path to the trained YOLO model.
    :param real_height_cm: Known real-world height of the  Coca-Cola bottle.
    :return: Bounding box height in pixels and estimated real-world height.
    """
    # Load the YOLO model
    model = YOLO(model_path)
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Perform object detection
    results = model(image)
    
    # Get the detected bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            
            # Calculate Pixel-to-CM ratio
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            # print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
            # print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
            
            return height_pixels, estimated_real_height
    
    print("No Coca-Cola bottle detected.")
    return None, None

# Example Usage
image_path = "image_folder/image10.jpeg"  # Change to your image path
model_path = "best (8).pt"  # Path to your trained YOLO model
height_pixels, estimated_real_height =detect_coke_bottle(image_path, model_path)

print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")


0: 800x608 2 750ml cokes, 387.4ms
Speed: 4.1ms preprocess, 387.4ms inference, 1.2ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.06 pixels
Estimated Real-World Height: 24.00 cm


In [3]:
def load_model(model_path):
    """Load the trained YOLO model"""
    return YOLO(model_path)

def detect_objects(model, image_path):
    """Detect objects in the image using the YOLO model"""
    image = cv2.imread(image_path)
    results = model(image)
    return results, image  # Return both detection results and image

def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

def draw_detections(image, propeller_boxes, floor_box):
    """Draw bounding boxes around detected propellers and floor with labels"""
    for i, (x_min, y_min, x_max, y_max) in enumerate(propeller_boxes):
        label = f"Propeller {i+1}"
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green for propellers
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    if floor_box:
        x_min, y_min, x_max, y_max = floor_box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Red for floor
        cv2.putText(image, "Floor", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    return image

In [4]:
def calculate_height(propeller_y, floor_y, calibration_obj=None):
    """
    Calculates the real-world height from the floor to each propeller.
    Uses a calibration object if available.
    """
    if propeller_y is None or floor_y is None:
        print("Propeller or Floor not detected!")
        return None
    
    pixel_height = propeller_y - floor_y  # Distance in pixels
    
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        real_propeller_height = pixel_height * pixel_to_cm_ratio
        return real_propeller_height
    
    return pixel_height  # Returns pixels if no calibration object

In [5]:
# Example usage
import os
model_floor_path = "best (5).pt"  # Model detecting floor, ceiling, wall
model_propeller_path = "best (9).pt"  # Model detecting propellers

calibration_obj = (estimated_real_height,height_pixels) 

# image_path = "Propellers test/IMG_3103.JPEG"  # Input image
output_folder = "Detected_Images"  # Folder to save images
os.makedirs(output_folder, exist_ok=True)  # Create folder if not exists

# Load YOLO models
floor_model = load_model(model_floor_path)
propeller_model = load_model(model_propeller_path)

# Detect floor using model best(5).pt
floor_results, image = detect_objects(floor_model, image_path)
floor_y_top, floor_box = get_floor_coordinates(floor_results)

# Detect propellers using model best(9).pt
propeller_results, _ = detect_objects(propeller_model, image_path)
propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)

if not propeller_boxes or floor_y_top is None:
    print(f"No propellers or floor detected in {image_path}")
else:
    print(f"\nImage: {image_path}")
    for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
        pixel_height = propeller_y_bottom - floor_y_top
        real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
        
        print(f"  Propeller {i+1}:")
        print(f"    - Pixel Height from Floor: {pixel_height} pixels")  
        print(f"    - Real-World Height from Floor: {real_height:.2f} cm")

    # Draw detected objects on image
    output_image = draw_detections(image, propeller_boxes, floor_box)

    # Display the output image
    cv2.imshow("Detected Objects", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the output image
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, output_image)
    print(f"Output saved at: {output_path}")


0: 640x480 1 ceiling, 4 floors, 5 wallss, 377.2ms
Speed: 2.4ms preprocess, 377.2ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 (no detections), 359.6ms
Speed: 5.6ms preprocess, 359.6ms inference, 0.5ms postprocess per image at shape (1, 3, 800, 608)
No propellers or floor detected in image_folder/image10.jpeg
